# Parallelized version

In [1]:
import math
import random
import numpy as np
import time

## Initializing spark

In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
cores = 8
conf = SparkConf().set("spark.executor.cores", cores).setAppName("appName").setMaster("local")
sc = SparkContext(conf=conf)

22/11/24 12:32:52 WARN Utils: Your hostname, Alexs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.244 instead (on interface en0)
22/11/24 12:32:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/24 12:32:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.app.startTime', '1669289572737'),
 ('spark.driver.port', '59212'),
 ('spark.app.name', 'appName'),
 ('spark.executor.id', 'driver'),
 ('spark.rdd.compress', 'True'),
 ('spark.executor.extraJavaOpt

## Functions

### Auxiliar functions

In [5]:
def trainTestSplit(dataset):
    train = dataset.filter(lambda x: x[2]==1)
    train = train.map(lambda x: (x[0], x[1]))
    test = dataset.filter(lambda x: x[2]==0)
    test = test.map(lambda x: (x[0], x[1]))
    return (train, test)

In [6]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [7]:
def rdd_cost_function(RDD_Xyyhat, lambda_ref, m, w):
    y = RDD_Xyyhat[1]
    y_hat = RDD_Xyyhat[2]
    temp = (-1/m) * (
        y * np.log(y_hat) + (1 - y)
        * np.log(1 - y_hat)
    )
    cost = temp + (lambda_ref/(2*m))*sum([i*i for i in w.value])
    return cost

In [8]:
def str_to_number_list(line):
    numbers = line.replace(" ","").split(",")
    numbers = [float(number) for number in numbers]
    
    # Separate X and y
    numbers = (numbers[:-1], numbers[-1])
    
    return numbers

In [9]:
def sum_list_values(list1, list2):
    return [n1 + n2 for n1, n2 in zip(list1, list2)]

In [10]:
def sum_minus_mean_squared(numbers):
    return [(number-means.value[x])**2 for x,number in enumerate(numbers)]

In [11]:
def rdd_mean_by_column(RDD_Xy, m):
    means = RDD_Xy.map(lambda x: x[0]).reduce(sum_list_values)
    means = [x/m for x in means]
    return means

In [12]:
def rdd_mean_by_column(RDD_Xy, m):
    means = RDD_Xy.map(lambda x: x[0]).reduce(sum_list_values)
    means = [x/m for x in means]
    return means

In [13]:
def rdd_std_by_column(RDD_Xy, m):
    stds = RDD_Xy.map(lambda x: x[0]).map(sum_minus_mean_squared).reduce(sum_list_values)
    stds = [math.sqrt(x/m) for x in stds]
    return stds

In [14]:
def multiply_RDDXy_by_w(Xy):
    global b
    tot = 0
    for xi, wi in zip(Xy[0], w.value):
        tot += xi * wi
    tot += b
    return (Xy[0], Xy[1], sigmoid(tot))

In [15]:
def calculate_dw(RDD_Xyyhat):
    dw = []
    for x in RDD_Xyyhat[0]:
        dw.append((RDD_Xyyhat[2]-RDD_Xyyhat[1])*x)
    return dw

### Mandatory functions

In [16]:
def readFile(filename):
    dataset = sc.textFile(filename)
    dataset = dataset.map(str_to_number_list)
    return dataset.sample(False, 1)

In [17]:
def normalize(RDD_Xy):
    m = RDD_Xy.count()
    global means
    means = sc.broadcast(rdd_mean_by_column(RDD_Xy, m))
    stds = sc.broadcast(rdd_std_by_column(RDD_Xy, m))
    norm_rdd = RDD_Xy.map(lambda values: ([(x - mean) / std for x, mean, std in zip(values[0], means.value, stds.value)], values[1]))
    return norm_rdd

In [18]:
def train(RDD_Xy, iterations, learning_rate, lambda_reg):
    global b
    global m
    m = RDD_Xy.count()
    n = len(RDD_Xy.take(1)[0][0])
    global w
    w_temp = w.value.copy()
    for it in range(iterations):
        RDD_Xyyhat = RDD_Xy.map(multiply_RDDXy_by_w)
        print(f"Cost for it {it}:", RDD_Xyyhat.map(lambda x: rdd_cost_function(x, lambda_reg, m, w)).reduce(lambda x,y: x+y))
        dw=[0 for i in range(0,n)]
        for cl in range(n):
                X_cl = RDD_Xyyhat.map(lambda x: (x[2]-x[1])*x[0][cl]).reduce(lambda x, y: x+y)
                dw[cl] = (1/m)*(X_cl)+(lambda_reg/m)*w_temp[cl]
                w_temp[cl] -= learning_rate * dw[cl]
        w = sc.broadcast(w_temp)
        db = (1/m)*RDD_Xyyhat.map(lambda x: x[2]-x[1]).reduce(lambda x, y: x+y)
        b -= learning_rate * db
    return w, b

In [ ]:
def accuracy(ws, b, RDD_Xy):
    total = RDD_Xy.count()
    y_and_y_hat = RDD_Xy.map(lambda x: (x[1], predict(ws, x[0], b)))
    correct = y_and_y_hat.map(lambda x: 1 if x[0]==x[1] else 0).reduce(lambda x, y: x+y)
    return correct/total

In [20]:
def predict(w, X, b):
    tot = 0
    for xi, wi in zip(X, w.value):
        tot += xi * wi
    tot += b
    val = sigmoid(tot)
    if (val>=0.5):
        return 1.0
    return 0.0

## Testing

In [21]:
RDD_Xy = readFile("../data/botnet_tot_syn_l.csv")
RDD_Xy.count()

1000000

In [22]:
RDD_Xy_normalized = normalize(RDD_Xy)

In [23]:
RDD_Xy_presplit = RDD_Xy_normalized.map(lambda x: (x[0], x[1], 1 if random.random() < 0.75 else 0))
train_data, test_data = trainTestSplit(RDD_Xy_presplit)

#### Initialize some stuff before training

In [24]:
np.random.seed(33)

global b  
b = 0

global n
n = len(RDD_Xy.take(1)[0][0])

global w
w = sc.broadcast(np.random.rand(n))

#### Training

In [25]:
w_final, b = train(train_data, 10, 1.5, 0)

Cost for it 0: 1.4447913254377145


Cost for it 1: 0.7355278867442587


ERROR:root:KeyboardInterrupt while sending command.====>            (7 + 1) / 9]
Traceback (most recent call last):
  File "/Users/alex/.pyenv/versions/3.7.10/envs/sparkk/lib/python3.7/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/alex/.pyenv/versions/3.7.10/envs/sparkk/lib/python3.7/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/alex/.pyenv/versions/3.7.10/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

#### Accuracy

In [ ]:
acc = accuracy(w_final, b, test_data)

## Experiments

In [22]:
elapsed_times = {}
start = time.time()

for cores in range(1,9,1):
    sc.stop()
    conf = SparkConf().set("spark.executor.cores", cores).setAppName("appName").setMaster("local")
    sc = SparkContext(conf=conf)
    
    print(f"---------- Starting execution with {cores} cores ----------")
    
    # ---------- Execution ----------
    RDD_Xy = readFile("../data/botnet_tot_syn_l.csv")
    RDD_Xy_normalized = normalize(RDD_Xy)
    RDD_Xy_presplit = RDD_Xy_normalized.map(lambda x: (x[0], x[1], 1 if random.random() < 0.75 else 0))
    train_data, test_data = trainTestSplit(RDD_Xy_presplit)
    
    np.random.seed(33)
    global b  
    b = 0
    global n
    n = len(RDD_Xy.take(1)[0][0])
    global w
    w = sc.broadcast(np.random.rand(n))
    w_final, b = train(train_data, 3, 1.5, 0)
    acc = accuracy(w_final, b, test_data)
    print(f"The accuracy for the test set is: {acc}")
    
    # ---------- Execution ----------

    end = time.time()
    print(f"---------- Finished execution with {cores} cores ----------")
    elapsed_time = end - start
    elapsed_times[cores] = elapsed_time
    print(f"Elapsed time for cores {cores} is {elapsed_time} seconds")

---------- Starting execution with 1 cores ----------


Cost for it 0: 1.4426934241063776


Cost for it 1: 0.7367914844030778


Cost for it 2: 0.45089955383920527


Accuacy 86.5564716241095%
The accuracy for the test set is: 0.865564716241095
---------- Finished execution with 1 cores ----------
522.6194360256195
---------- Starting execution with 2 cores ----------


...

Accuacy 86.49126455906821%
The accuracy for the test set is: 0.8649126455906821
---------- Finished execution with 7 cores ----------
4263.749477863312
---------- Starting execution with 8 cores ----------


Cost for it 0: 1.4461814799528798


Cost for it 1: 0.7357183021970781


Cost for it 2: 0.4505616059294015


Accuacy 86.7265820958914%
The accuracy for the test set is: 0.867265820958914
---------- Finished execution with 8 cores ----------
4784.268172979355


In [23]:
elapsed_times

{1: 522.6194360256195,
 2: 1362.290090084076,
 3: 2190.6708841323853,
 4: 2710.363261938095,
 5: 3221.615458011627,
 6: 3742.4106187820435,
 7: 4263.749477863312,
 8: 4784.268172979355}